In [7]:
#stolen from https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch08/ch08.ipynb
import pyprind
import pandas as pd
import os

basepath = '/Users/Users/arakipovic/Dropbox/ML'
basepath = './aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:03:03


In [10]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./movie_data.csv', index=False)

In [11]:
df = pd.read_csv('./movie_data.csv')
df.head(3)

,review,sentiment
0,"Election is a Chinese mob movie, or triads in ...",1
1,I was just watching a Forensic Files marathon ...,0
2,Police Story is a stunning series of set piece...,1


In [22]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#df = pd.read_csv('./movie_data.csv')
count = CountVectorizer()
docs = df['review'][0:2]

bag = count.fit_transform(docs)
print(count.vocabulary_)
print(bag.toarray())

tfdtf = TfidfTransformer()
np.set_printoptions(precision=2)
print(tfdtf.fit_transform(count.fit_transform(docs)).toarray())

{'election': 66, 'is': 103, 'chinese': 44, 'mob': 128, 'movie': 132, 'or': 146, 'triads': 200, 'in': 98, 'this': 191, 'case': 40, 'every': 69, 'two': 205, 'years': 224, 'an': 11, 'held': 89, 'to': 196, 'decide': 54, 'on': 143, 'new': 137, 'leader': 115, 'and': 13, 'at': 20, 'first': 76, 'it': 104, 'seems': 170, 'toss': 198, 'up': 207, 'between': 32, 'big': 33, 'tony': 197, 'leung': 117, 'ka': 109, 'fai': 70, 'as': 18, 'know': 111, 'him': 93, 'the': 187, 'other': 147, 'lok': 120, 'simon': 175, 'yam': 222, 'who': 215, 'was': 209, 'judge': 107, 'full': 79, 'contact': 48, 'though': 192, 'once': 144, 'wins': 219, 'refuses': 162, 'accept': 4, 'choice': 45, 'goes': 81, 'whatever': 212, 'lengths': 116, 'he': 88, 'can': 39, 'secure': 169, 'recognition': 161, 'unlike': 206, 'any': 14, 'asian': 19, 'film': 75, 'watch': 210, 'featuring': 72, 'gangsters': 80, 'one': 145, 'not': 139, 'action': 6, 'has': 86, 'its': 105, 'bloody': 35, 'moments': 129, 'when': 213, 'necessary': 136, 'goodfellas': 82, 'b

In [24]:
import re

def preprocessor(text):
    text = re.sub('<[^<]+?>', '', text)
    emoticons = re.findall('(?::|;||=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ''.join(emoticons).replace('-', '')
    return text

In [25]:
df['review'] = df['review'].apply(preprocessor)

In [26]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [28]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arakipovic/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


X_train, X_test, y_train, y_test = train_test_split(df['review'].values, df['sentiment'].values, test_size=0.1)
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None], 
               'vect__use_idf': [False],
               'vect__norm': [None],
               'clf__penalty': ['l1', 'l2'], 
               'clf__C': [1.0, 10.0, 100.0]}
]

lr_tfidf = Pipeline([('vect', tfidf), 
                     ('clf', LogisticRegression(random_state=0))])

gs = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 21.9min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'vect__ngram_range': [(1, 1)], 'vect__stop_words': [['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', '...se_idf': [False], 'vect__norm': [None], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=1

In [49]:
print('Best params: {}'.format(gs.best_params_))
print('best score {}'.format(gs.best_score_))
clf = gs.best_estimator_
print('test acc: {}'.format(clf.score(X_test, y_test)))

Best params: {'clf__C': 1.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__norm': None, 'vect__stop_words': None, 'vect__use_idf': False}
best score 0.8884
test acc: 0.889


In [53]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

porter = PorterStemmer()

def tokenizer_porter(text):
    return [word for word in text.split()]

stop = stopwords.words('english')

def tokenizer_porter_stop_remove(text):
    return [porter.stem(word) for word in text.split() if word not in stop]


def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label
            
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            doc, label = next(doc_stream)
            docs.append(doc)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y


vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, 
                         tokenizer=tokenizer_porter)
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='./movie_data.csv')

pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
        
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()
    

X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('acc test: {}'.format(clf.score(X_test, y_test)))

clf = clf.partial_fit(X_test, y_test)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:10


acc test: 0.8212
